In [1]:
import tensorflow as tf
import numpy as np
import os
from fxpmath import Fxp

## Cargando el modelo (mismo codigo que usado durante el desarrollado en tensorflow)

In [2]:
def Quantization_layer(tensor, Quantization = True,signed = True, word_size = 12, frac_size = 6):
    
    factor = 2.0**frac_size
    
    if Quantization:
        return tf.round(tensor*factor) / factor             #Quantization, assuming no overflow
    else:
        return tensor                                       #Simple Bypass

In [3]:
def build_model(input_layer, Quantization = True, signed = True, word_size = 12, frac_size = 6 ):
    Arguments = {'Quantization':Quantization, 'signed':signed, 'word_size':word_size, 'frac_size':frac_size}
    QInp      = tf.keras.layers.Lambda(Quantization_layer, name="QInp",  arguments = Arguments )(input_layer)
    #Conv Block
    Conv1   = tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, input_shape=(28,28,1), padding='same', name= 'Conv1')(QInp)
    QConv1  = tf.keras.layers.Lambda(Quantization_layer, name="QConv1",  arguments = Arguments )(Conv1)
    Act1    = tf.keras.activations.tanh(QConv1)
    QAct1   = tf.keras.layers.Lambda(Quantization_layer, name="QAct1",   arguments = Arguments )(Act1)
    AvgPool1= tf.keras.layers.AveragePooling2D(name='AvgPool1')(QAct1)
    #Conv Block
    Conv2   = tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding='valid',name='Conv2')(AvgPool1)
    QConv2  = tf.keras.layers.Lambda(Quantization_layer, name="QConv2",  arguments = Arguments )(Conv2)
    Act2    = tf.keras.activations.tanh(QConv2)
    QAct2   = tf.keras.layers.Lambda(Quantization_layer, name="QAct2",   arguments = Arguments )(Act2)
    AvgPool2= tf.keras.layers.AveragePooling2D(name='AvgPool2')(QAct2)
    Flatten = tf.keras.layers.Flatten(name='Flatten')(AvgPool2)
    #Dense Block
    Dense1  = tf.keras.layers.Dense(units=120, name='Dense1')(Flatten)
    QDense1 = tf.keras.layers.Lambda(Quantization_layer, name="QDense1", arguments = Arguments )(Dense1)
    Act3    = tf.keras.activations.tanh(QDense1)
    QAct3   = tf.keras.layers.Lambda(Quantization_layer, name="QAct3",   arguments = Arguments )(Act3)
    #Dense Block
    Dense2  = tf.keras.layers.Dense(units=84, name='Dense2')(QAct3)
    QDense2 = tf.keras.layers.Lambda(Quantization_layer, name="QDense2", arguments = Arguments)(Dense2)
    Act4    = tf.keras.activations.tanh(QDense2)
    QAct4   = tf.keras.layers.Lambda(Quantization_layer, name="QAct4",   arguments = Arguments)(Act4)
    #Output Block
    Out     = tf.keras.layers.Dense(units=10,name='Output')(QAct4)
    QOut    = tf.keras.layers.Lambda(Quantization_layer, name="QOut",    arguments = Arguments)(Out)
    Act5    = tf.keras.activations.softmax(QOut)
    QAct5   = tf.keras.layers.Lambda(Quantization_layer, name="QSoftmax",arguments = Arguments)(Act5)
    
    return QAct5

In [4]:

Qinput_layer  = tf.keras.Input((28, 28, 1))
Qoutput_layer = build_model(Qinput_layer, Quantization = True, word_size = 16, frac_size = 8)

QLenet = tf.keras.Model(inputs=Qinput_layer, outputs=Qoutput_layer)

In [5]:
cwd = os.getcwd()
Wgt_dir = os.path.join(cwd,'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir,'Weights')
QLenet.load_weights(Wgt_dir)

In [6]:
def Quantization(List, Quantization = True, signed = True, word_size = 12, frac_size = 6):
    factor = 2.0**frac_size
    return tf.round(np.array(List)*factor) / factor             #Quantization, assuming no overflow

for layer in QLenet.layers:
    weights = layer.get_weights()
    if weights:                     # Layer with weights
        # Quantization of Weights and Bias 
        Qweights    = [None,None]
        Qweights[0] = Quantization(weights[0], word_size = 12, frac_size = 3)
        Qweights[1] = Quantization(weights[1], word_size = 12, frac_size = 3)
        layer.set_weights(Qweights)

## Datos de Simulacion

In [7]:
#Referente a la cuantizacion
Sign_bits = 1
Int_bits  = 7
Frac_bits = 8
Word_bits = Sign_bits + Int_bits + Frac_bits

# Referente a los buffers
IOBuffer_size = 4704*Word_bits



In [8]:
IOBuffer_1 = np.zeros(IOBuffer_size,dtype=int)
IOBuffer_2 = np.zeros(IOBuffer_size,dtype=int)

## Cargando los datos en el buffer

In [9]:
# Loading Data
(_, _), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_test = x_test/255.0

In [10]:
def Quantization(array, Quantization = True):
    
    factor = 2.0**Frac_bits
    
    if Quantization:
        return np.round(array*factor) / factor             #Quantization, assuming no overflow
    else:
        return array                                       #Simple Bypass 

In [11]:
sim_x_test = Quantization(x_test)

In [12]:
# la imagen es guardada pixel a pixel de izquierda a derecha, de arriba a abajo.
def Load_Image_grayscale(buffer, data):
    index = 0
    for row in data:
        for pixel in row:
            binary_pixel = Fxp(pixel,True,Word_bits,Frac_bits)
            binary_pixel = binary_pixel.bin()
            for bit in binary_pixel:
                buffer[index] = bit
                index = index + 1

In [13]:
Load_Image_grayscale(IOBuffer_1,sim_x_test[0])

## Funcion para obtener activaciones de la red

In [15]:
x_test = tf.expand_dims(x_test,axis=-1)

In [16]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.backend import eager_learning_phase_scope

# Function to get outputs from each layer.
def get_all_outputs(model, input_data, learning_phase=False):
    outputs = [layer.output for layer in model.layers] # exclude Input
    layers_fn = K.function([model.input, K.symbolic_learning_phase()], outputs)
    return layers_fn([input_data, learning_phase])

## Funcion para escribir en buffers

In [17]:
def write_conv_output(data,offset,buffer):
    binary_data = Fxp(data,True,Word_bits,Frac_bits)
    binary_data = binary_data.bin()
    index = offset
    for bit in binary_data:
        buffer[index] = bit
        index = index + 1

## Estadisticas de buffers

In [18]:
stats_IOBuffer_1 = np.zeros((4,IOBuffer_size),dtype=int)
stats_IOBuffer_2 = np.zeros((4,IOBuffer_size),dtype=int)

In [19]:
def buffer_stadistics(data_buffer,stats_buffer,n_cycles):
    # (last value, # cambios logicos, # ciclos almacenando 0, # ciclos almacenando 1)
    index = 0
    while index < len(data_buffer):
        if data_buffer[index] != stats_buffer[0,index]:
            stats_buffer[1,index] = stats_buffer[1,index]  + 1
        if data_buffer[index] == 0:
            stats_buffer[2,index] = stats_buffer[2,index]  + n_cycles
        if data_buffer[index] == 1:
            stats_buffer[3,index] = stats_buffer[3,index]  + n_cycles
        stats_buffer[0,index] = data_buffer[index]
        index = index + 1

## Simulacion

In [23]:
print("inicia simulacion")
cycles = 0                  # Contador de ciclos
print("Capa Convolucional")
# Offsets para escritura en memoria
map_offset    = 28*28*16
row_offset    = 28*16
column_offset = 16
# Indices de lectura.
row     = 0
column  = 0
act_map = 0
outputs = get_all_outputs(QLenet, tf.expand_dims(x_test[0],axis=0))
layer_outputs = outputs[5]  # activaciones de la capa convolucional
outs_per_cycle = 6          # Numero de activaciones que se calculan por ciclo.
# Primer ciclo recorre la capa convolucional, el segundo las convoluciones de cada ciclo.
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,row,column,act_map]
        write_conv_output(out,act_map*map_offset + row*row_offset + column*column_offset, IOBuffer_2)
        if act_map == 5:
            act_map = -1
            column  = column + 1
        if column == 28:
            column  = 0
            row     = row + 1
        if row == 28:
            break
        act_map = act_map + 1
        counter = counter + 1
        if counter == outs_per_cycle:
            break
    # Luego de cada ciclo se revisa el buffer para actualizar las estadisticas.
    buffer_stadistics(IOBuffer_2,stats_IOBuffer_2,1)
    if row == 28:
        break
print("Ciclos procesados:",cycles)
print("Capa AvgPool")
# Analogo a la capa anterior
map_offset    = 14*14*16
row_offset    = 14*16
column_offset = 16
row     = 0
column  = 0
act_map = 0
layer_outputs = outputs[6]
outs_per_cycle = 24
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,row,column,act_map]
        write_conv_output(out,act_map*map_offset + row*row_offset + column*column_offset, IOBuffer_1)
        if act_map == 5:
            act_map = -1
            column  = column + 1
        if column == 14:
            column  = 0
            row     = row + 1
        if row == 14:
            break
        act_map = act_map + 1
        counter = counter + 1
        if counter == outs_per_cycle:
            break
    buffer_stadistics(IOBuffer_1,stats_IOBuffer_1,1)
    if row == 14:
        break
print("Ciclos procesados:",cycles)
print("Capa Convolucional")
map_offset    = 10*10*16
row_offset    = 10*16
column_offset = 16
row     = 0
column  = 0
act_map = 0
layer_outputs = outputs[10]
outs_per_cycle = 6
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,row,column,act_map]
        write_conv_output(out,act_map*map_offset + row*row_offset + column*column_offset, IOBuffer_2)
        if act_map == 15:
            act_map = -1
            column  = column + 1
        if column == 10:
            column  = 0
            row     = row + 1
        if row == 10:
            break
        act_map = act_map + 1
        counter = counter + 1
        if counter == outs_per_cycle:
            break
    buffer_stadistics(IOBuffer_2,stats_IOBuffer_2,1)
    if row == 10:
        break
print("Ciclos procesados:",cycles)
print("Capa AvgPool")
map_offset    = 5*5*16
row_offset    = 5*16
column_offset = 16
row     = 0
column  = 0
act_map = 0
layer_outputs = outputs[11]
outs_per_cycle = 24
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,row,column,act_map]
        write_conv_output(out,act_map*map_offset + row*row_offset + column*column_offset, IOBuffer_1)
        if act_map == 15:
            act_map = -1
            column  = column + 1
        if column == 5:
            column  = 0
            row     = row + 1
        if row == 5:
            break
        act_map = act_map + 1
        counter = counter + 1
        if counter == outs_per_cycle:
            break
    buffer_stadistics(IOBuffer_1,stats_IOBuffer_1,1)
    if row == 5:
        break
print("Ciclos procesados:",cycles)
print("Capa FC")
# en este caso dado que son muchas multiplicaciones las necesarias se hace lo inverso, se define cuantos ciclos se necesita por activacion.
neuron_offset = 16
neuron = 0
layer_outputs = outputs[16]
cycles_per_neuron = 3
while True:
    cycles = cycles + cycles_per_neuron
    out = layer_outputs[0,neuron]
    write_conv_output(out, neuron*neuron_offset, IOBuffer_2)
    neuron = neuron + 1
    buffer_stadistics(IOBuffer_2,stats_IOBuffer_2,cycles_per_neuron)
    if neuron == 120:
        break
print("Ciclos procesados:",cycles)
print("Capa FC")
neuron_offset = 16
neuron = 0
layer_outputs = outputs[20]
neurons_per_cycle = 2
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,neuron]
        write_conv_output(out, neuron*neuron_offset, IOBuffer_1)
        neuron = neuron + 1
        counter = counter + 1
        if counter == neurons_per_cycle or neuron == 84:
            break
    buffer_stadistics(IOBuffer_1,stats_IOBuffer_1,1)
    if neuron == 84:
        break
print("Ciclos procesados:",cycles)
print("Capa FC")
neuron_offset = 16
neuron = 0
layer_outputs = outputs[24]
neurons_per_cycle = 3
while True:
    cycles = cycles + 1
    counter = 0
    while True:
        out = layer_outputs[0,neuron]
        write_conv_output(out, neuron*neuron_offset, IOBuffer_2)
        neuron = neuron + 1
        counter = counter + 1
        if counter == neurons_per_cycle or neuron == 10:
            break
    buffer_stadistics(IOBuffer_2,stats_IOBuffer_2,1)
    if neuron == 10:
        break
print("Ciclos procesados:",cycles)

inicia simulacion
Capa Convolucional
Ciclos procesados: 784
Capa AvgPool
Ciclos procesados: 833
Capa Convolucional
Ciclos procesados: 1100
Capa AvgPool
Ciclos procesados: 1117
Capa FC
Ciclos procesados: 1477
Capa FC
Ciclos procesados: 1519
Capa FC
Ciclos procesados: 1523
